In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
import logging
import json
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
import scipy.sparse as sps
from sklearn.model_selection import KFold
from gensim.models.tfidfmodel import TfidfModel
from itertools import chain
from itertools import count
import nltk
from sklearn import linear_model
from sklearn import model_selection
from nltk.corpus import stopwords
import gc
from itertools import islice
import string

In [3]:
logging.basicConfig(level=logging.INFO)

In [287]:
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense, Dropout, Input, Lambda, Merge, merge, LSTM
from keras.optimizers import RMSprop
from keras import backend as K

In [5]:
data = pd.read_pickle("cdata/train_cleaned_nostop_vectorized.pickle")

In [6]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[-0.143764, 0.289036, 0.11826, 0.158218, 0.327...","[-0.0184767, 0.272587, 0.0296637, 0.212434, 0....",0
1,1,3,4,"[0.13491, 0.19703, 0.328434, -0.0068158, -0.30...","[-0.0288371, 0.082105, 0.211458, -0.0099432, -...",0
2,2,5,6,"[-0.0451945, 0.311742, -0.0901077, -0.123615, ...","[-0.397313, 0.354054, -0.113698, -0.194915, -0...",0
3,3,7,8,"[-0.04095, 0.193038, -0.328477, -0.0551063, -0...","[-0.0185197, 0.287688, 0.15372, 0.0815627, 0.1...",0
4,4,9,10,"[-0.139202, 0.206397, -0.0212343, -0.0150773, ...","[-0.165639, 0.0432734, 0.0210668, -0.0945684, ...",0


In [7]:
X = list(map(lambda x: np.stack((x[0],x[1])), zip(data.question1.values, data.question2.values)))
Y = data.is_duplicate.values
train_X, test_X, train_Y, test_Y = list(map(np.array, model_selection.train_test_split(X, Y, train_size=0.8)))
input_dim = 300
epochs = 20

In [279]:
"""def euclidean_distance(vects):
    x, y = vects
    return 1 - K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

"""
def dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def dot(vects):
    x, y = vects
    return (x-y)**2

# network definition
input_dim = input_dim
base_network = Sequential()
base_network.add(Dense(500, input_shape=(input_dim,), activation='relu'))
base_network.add(Dropout(0.1))
base_network.add(Dense(200, activation='relu'))
#base_network.add(Dropout(0.1))
#base_network.add(Dense(200, activation='relu'))

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(dot, output_shape=dist_output_shape)([processed_a, processed_b])

#distance = merge([processed_a, processed_b], mode="multiply")
print(distance.get_shape())

cosine = Sequential()
cosine.add(Dense(1, init = 'normal', activation = 'sigmoid', input_shape=(200,)))
answer = cosine(distance)

model = Model([input_a, input_b], answer)

# train
rms = RMSprop()

def log_loss(y_true, y_pred):
    return K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))

# loss='binary_crossentropy'
model.compile(loss='mean_squared_error', optimizer=rms)
#model.compile(loss='binary_crossentropy', optimizer=rms)

(?, 200)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, input_shape=(200,), kernel_initializer="normal", activation="sigmoid")`


In [280]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [281]:
model.fit([train_X[:, 0], train_X[:, 1]], train_Y,
          batch_size=128,
          epochs=10,
          validation_data=([test_X[:, 0], test_X[:, 1]], test_Y))


Train on 323432 samples, validate on 80858 samples
Epoch 1/10
323432/323432 [==============================] - 9s - loss: 0.1692 - val_loss: 0.1547

In [282]:
pred = model.predict([test_X[:, 0], test_X[:, 1]])

In [283]:
from sklearn import metrics

In [284]:
pred[pred<0.001] = 0.001
pred[pred>0.999] = 0.999
pred[pred==np.nan] = 0

In [286]:
metrics.f1_score(test_Y, pred[:,0])

ValueError: Can't handle mix of binary and continuous

In [285]:
np.mean(-(test_Y * np.log(pred[:,0]) + (1 - test_Y) * np.log(1 - pred[:,0])))

0.43009957594307135

200 200 200    dot  dense = 0.49

200 200 200    euclid  dense = 0.46

500 200        euclid dense = 0.43

500 500        cosine dense = 0.53

contrastive_loss  -> bullshit!  2.3

1000 200       cosine dense =  0.51

1000 200       euclid dense = overfit
1000 200       euclid dense =  overfit (higher dropout)


In [58]:
pred[:,0].max()

1.0

In [59]:
metrics.log_loss(test_Y, pred[:,0])

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


nan

0.00031622776

In [18]:
log_loss(test_Y, pred.T[0])

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


nan

In [46]:
model.fit([np.concatenate((train_X[:, 0],test_X[:, 0])), np.concatenate((train_X[:, 1], test_X[:, 1]))], np.concatenate((train_Y, test_Y)),
          batch_size=128,
          epochs=epochs,
          validation_data=([test_X[:, 0], test_X[:, 1]], test_Y))


Train on 404290 samples, validate on 80858 samples
Epoch 1/20
404290/404290 [==============================] - 10s - loss: 0.1391 - val_loss: 0.1350

In [47]:
def serialize(model, fname):
    f = open(fname, "w")
    f.write(model.to_json())
    model.save_weights(fname+"w")

In [48]:
serialize(model, "models/initial_siames")

In [37]:
def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

# compute final accuracy on training and test sets
pred = model.predict([train_X[:, 0], train_X[:, 1]])
tr_acc = compute_accuracy(pred, train_Y)
pred = model.predict([test_X[:, 0], test_X[:, 1]])
te_acc = compute_accuracy(pred, test_Y)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 70.96%
* Accuracy on test set: 71.11%


In [5]:
def deserialize(fname):
    model = model_from_json(open(fname).read())
    model.load_weights(fname+"w")
    return model

In [6]:
model2 = deserialize("models/initial_siames")

In [7]:
sharp = pd.read_pickle("cdata/test_cleaned_nostop_vectorized.pickle")
#sharp = np.array(map(lambda x: np.stack((x[0],x[1])), zip(data.question1.values, data.question2.values)))


In [11]:
preds = []
parts = 1
part_size = (sharp.shape[0]//parts)
for i in range(parts+1):
    print(i)
    print(i)
    work = sharp[i*part_size:][:part_size]
    sharp_array = np.array(list(map(lambda x: np.stack((x[0],x[1])), zip(work.question1.values, work.question2.values))))
    submit_predictions = model2.predict([sharp_array[:, 0], sharp_array[:,1]], verbose=1)
    preds.append([submit_predictions, work.test_id.values])

0
117289/117289 [==============================] - 3s     

In [26]:
predictions = np.array([[0]])
indices = np.array([0])
for x in preds:
    predictions = np.concatenate((predictions, x[0]))
    indices = np.concatenate((indices, x[1]))

array([      0,       0,       1, ..., 2345793, 2345794, 2345795])

In [31]:
sub = pd.DataFrame({'test_id': indices[1:], 'is_duplicate': predictions[1:].T[0]})
sub.to_csv('siames.csv', index=False)
sub.head()

,is_duplicate,test_id
0,0.431685,0
1,0.774310,1
2,0.843862,2
3,0.908784,3
4,0.606570,4
